### Following from: https://www.youtube.com/watch?v=PaCmpygFfXo

### Ideas to follow up:
TODO

In [ ]:
import torch
import torch.nn.functional as F


In [ ]:
words = open("names.txt", "r").read().splitlines()

In [ ]:
max(len(w) for w in words)

In [ ]:
words[:10]

In [ ]:
b = {} # bigram count
for w in words:
    chs = ["<S>"] + list(w) + ["<E>"]
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

In [ ]:
sorted(b.items(), key=lambda kv:kv[1], reverse=True)[:10]

In [ ]:


i2s

In [ ]:
N = torch.zeros(27, 27)

for w in words:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        idx1 = s2i[ch1]
        idx2 = s2i[ch2]
        N[idx1, idx2] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16, 16))
plt.imshow(N, cmap="Blues")

for i in range(27):
    for j in range(27):
        chstr = i2s[i] + i2s[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color="gray")
        plt.text(j, i, f"{N[i, j].item()}", ha="center", va="top", color="gray")
plt.axis("off")

In [ ]:
prob = N[0].float()
prob /= prob.sum()
prob

In [ ]:
g = torch.Generator().manual_seed(2147483647)

p = torch.rand(3, generator=g)
p = p/p.sum()

In [ ]:
torch.multinomial(p, num_samples=20, replacement=True, generator=g)

In [ ]:
g = torch.Generator().manual_seed(2147483647)
idx = torch.multinomial(prob, num_samples=1, replacement=True, generator=g).item()
i2s[idx]

In [ ]:
# TODO: read about broadcasting
P = (N + 1).float()
P /= P.sum(dim=1, keepdim=True)


In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = []
    idx = 0
    while True:
        # p = torch.ones(27) / 27.0
        idx = torch.multinomial(P[idx], num_samples=1, replacement=True, generator=g).item()
        out.append(i2s[idx])
        if idx == 0:
            break
    print("".join(out))

In [ ]:
# idea: trigram; n-gram;

In [ ]:
log_likelyhood = 0.0
n = 0
for w in ["mircea"]:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        idx1, idx2 = s2i[ch1], s2i[ch2] 
        logprob = torch.log(P[idx1, idx2]).item()
        log_likelyhood += logprob
        n += 1
        print(f"{ch1}{ch2} : {P[idx1, idx2].item():.3f} {logprob:.3f}")

nll = - log_likelyhood
print(f"negative_log_likelyhood: {nll:.3f}")
print(f"{nll / n:.3f}")

In [ ]:
# TODO: maximum likelihood estimation (wiki)
# log-likelihood

In [ ]:
# create training set of bigrams
xs, ys = [], []

for w in words:
    chs = ["."] + list(w) + ["."]	
    for ch1, ch2 in zip(chs, chs[1:]):
        idx1, idx2 = s2i[ch1], s2i[ch2]
        # print(ch1, ch2)
        xs.append(idx1)
        ys.append(idx2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

num = xs.nelement()
print(f"number of examples: {num}")

In [ ]:
# torch.nn.functional.one_hot

In [ ]:
x_enc = F.one_hot(xs, num_classes=27).float()
x_enc.shape
plt.imshow(x_enc)

In [ ]:
W = torch.randn((27, 27))
# (5, 27) @ (27, 27) = (5, 27)
(x_enc @ W)[3,13]

In [ ]:
logits = x_enc @ W # log-counts
counts = logits.exp()

prob = counts / counts.sum(dim=1, keepdim=True)


In [ ]:
prob[0]

In [ ]:
xs

In [ ]:
ys

In [ ]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g)
x_enc = F.one_hot(xs, num_classes=27).float()
logits = x_enc @ W # log-counts
# softmax
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdim=True)

In [ ]:
nlls = torch.zeros(5)
for i in range(5):
    x = xs[i].item()
    y = ys[i].item()
    print("-----------------------------------")
    print(f"Example: {i2s[x]} -> {i2s[y]}, indices {x} -> {y}")
    p = probs[i, y]
    logp = torch.log(p)
    print(f"Log likelyhood: {logp:.3f}")
    nlls[i] = -logp
print(f"Average log likelyhood: {nlls.mean().item():.3f}")

In [ ]:
loss = -probs[torch.arange(5), ys].log().mean() # nll; idx1: 0..5; idx2: ys

In [ ]:
# g = torch.Generator().manual_seed(2147483647)
# W = torch.randn((27, 27), generator=g, requires_grad=True)

# forward pass
x_enc = F.one_hot(xs, num_classes=27).float()
logits = x_enc @ W # log-counts
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdim=True)
loss = -probs[torch.arange(5), ys].log().mean() # nll; idx1: 0..5; idx2: ys


In [ ]:
# backward pass
W.grad = None # mored efficient than W.zero_grad() ???

loss.backward()

In [ ]:
W.data += -0.1 * W.grad


In [ ]:
print(loss.item())

In [ ]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
# gradient descent

n_iter = 100
num_elements = xs.nelement()
learning_rate = 50

for k in range(n_iter):
    # forward pass
    x_enc = F.one_hot(xs, num_classes=27).float()
    logits = x_enc @ W # log-counts
    counts = logits.exp()
    probs = counts / counts.sum(dim=1, keepdim=True)
    loss = -probs[torch.arange(num_elements), ys].log().mean() + 0.001 * (W ** 2).mean() # nll; idx1: 0..5; idx2: ys
    if k % 10 == 0:
        print(loss.item())

    # backward pass
    W.grad = None 
    loss.backward()

    # update W:
    W.data += -learning_rate * W.grad

In [ ]:
(W **2).sum()

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(20):
    out = []
    ix = 0
    while True:
        x_enc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = x_enc @ W 
        counts = logits.exp()
        probs = counts / counts.sum(dim=1, keepdim=True)
        ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        out.append(i2s[ix])
        if ix == 0:
            break
    print("".join(out))